In [1]:
import requests
from bs4 import BeautifulSoup as bs

# BeautifulSoup 활용 

## 1 CGV 차트 가져오기

In [15]:
url = 'http://www.cgv.co.kr/movies/'
response = requests.get(url)
soup = bs(response.text,'html.parser')

In [16]:
tags = soup.select('strong.title')
tags[:10]

[<strong class="title">아마존 활명수</strong>,
 <strong class="title">베놈: 라스트 댄스</strong>,
 <strong class="title">롱레그스</strong>,
 <strong class="title">날씨의 아이</strong>,
 <strong class="title">리틀 엠마</strong>,
 <strong class="title">보통의 가족</strong>,
 <strong class="title">쏜애플 불구경 2024 라이브 콘서트 필름</strong>,
 <strong class="title">최소한의 선의</strong>,
 <strong class="title">대도시의 사랑법</strong>,
 <strong class="title">임영웅ㅣ아임 히어로 더 스타디움</strong>]

In [18]:
tags2 = soup.find_all(name = 'strong', attrs = {'class':'title'})
tags2[:10]

[<strong class="title">아마존 활명수</strong>,
 <strong class="title">베놈: 라스트 댄스</strong>,
 <strong class="title">롱레그스</strong>,
 <strong class="title">날씨의 아이</strong>,
 <strong class="title">리틀 엠마</strong>,
 <strong class="title">보통의 가족</strong>,
 <strong class="title">쏜애플 불구경 2024 라이브 콘서트 필름</strong>,
 <strong class="title">최소한의 선의</strong>,
 <strong class="title">대도시의 사랑법</strong>,
 <strong class="title">임영웅ㅣ아임 히어로 더 스타디움</strong>]

In [14]:
for idx, tag in enumerate(tags,start=1):
    print(f'{idx}위 : {tag.text}')

1위 : 아마존 활명수
2위 : 베놈: 라스트 댄스
3위 : 롱레그스
4위 : 날씨의 아이
5위 : 리틀 엠마
6위 : 보통의 가족
7위 : 쏜애플 불구경 2024 라이브 콘서트 필름
8위 : 최소한의 선의
9위 : 대도시의 사랑법
10위 : 임영웅ㅣ아임 히어로 더 스타디움
11위 : 오후 네시
12위 : 너의 색
13위 : 명탐정 코난-100만 달러의 펜타그램
14위 : 공작새
15위 : 하이라이트: 라이츠 고 온, 어게인 인 시네마
16위 : 극장판 하이큐!! 쓰레기장의 결전
17위 : 노트북
18위 : 하와이 연가
19위 : 베테랑2


## 2 네이버 환율 가져오기

In [6]:
url = 'https://finance.naver.com/marketindex/exchangeList.naver'
response = requests.get(url)
soup = bs(response.text,'html.parser')

In [7]:
# 국가
# 미국 body > div > table > tbody > tr:nth-child(1) > td.tit > a
# 유렵 body > div > table > tbody > tr:nth-child(2) > td.tit > a
# 일본 body > div > table > tbody > tr:nth-child(3) > td.tit > a
rows = soup.select('body > div > table > tbody > tr') # 가로(행)
# rows

In [8]:
for row in rows:
    title = row.select_one('a').text.strip()
    # print(title)
    sale = row.select_one('td.sale').text
    # print(sale)

In [11]:
# print('나라 이름 / 환율')
# print('=' * 25)
for row in rows:
    title = row.select_one('a').text.strip()
    sale = row.select_one('td.sale').text
    # print(f'{title} / {sale}')
    # print('-' * 25)

## 2 크롤링을 이용한 환율 계산기(함수로 구현)

### 다른 나라의 통화를 원으로 계산

In [11]:
def get_exchange_rate(menu):
    url = 'https://finance.naver.com/marketindex/exchangeList.naver'
    response = requests.get(url)
    soup = bs(response.text,'html.parser')
    exchange_rates = soup.select('body > div > table > tbody > tr > td.sale') # 모든 환율
    exchange_rate = exchange_rates[menu-1].text.replace(',','')
    exchange_rate = float(exchange_rate)
    return exchange_rate

In [12]:
menu = int(input('''원하시는 국가를 선택해주세요!
                 ============
                 1. 미국
                 2. 유럽
                 3. 일본
                 4. 중국
                 =============
                 '''))
unit = ['달러','유로','엔','위안']
money = int(input(f'환율 계산할 금액을 입력해주세요. (단위 : {unit[menu-1]}) : '))

if menu == 3: # 일본을 선택했다면? 네이버에서는 100엔 기준 금액 제공
    result_money = get_exchange_rate(menu) * (money // 100)
else:
    result_money = get_exchange_rate(menu) * money

print(f'{money} {unit[menu-1]}은(는) 원으로 환전하면 {result_money:.1f}원입니다.')

100 달러은(는) 원으로 환전하면 138570.0원입니다.


In [13]:
# 원을 다른 나라 통화로 계산
menu = int(input('''원하시는 국가를 선택해주세요!
                 ============
                 1. 미국
                 2. 유럽
                 3. 일본
                 4. 중국
                 =============
                 '''))
money = int(input(f'환전할 금액을 입력해주세요. (단위 : 원) : '))
unit = ['달러','유로','엔','위안']
if menu == 3: # 일본을 선택했다면? 네이버에서는 100엔 기준 금액 제공
    result_money = (money // 100) / get_exchange_rate(menu)
else:
    result_money = money / get_exchange_rate(menu) 

print(f'{money} 원을 {unit[menu-1]}로 환전하면 {result_money:.1f}{unit[menu-1]}입니다.')

500000 원을 엔로 환전하면 5.5엔입니다.
